## Importing Dependencies

In [2]:
import requests
from htmldate import find_date
from datetime import datetime
import requests
import re
from bs4 import BeautifulSoup
import ssl
from tqdm import tqdm
from textblob import TextBlob
import openai
import multiprocessing
from time import time
import pandas as pd
import numpy as np 

from html_extractor import *
from get_suburls import *
from openai_func import *
from get_date import *
from parallel import *
from mongo_utils import import_from_mongo, save_to_mongo

from time import sleep

from keyword_extraction import keyword_extractor_paragraph as kep


from pymongo import MongoClient
import certifi
ca = certifi.where()

## Extracting sub urls

In [4]:
urls_list = ["https://www.khaleejtimes.com" , "https://www.indiatoday.in"]
urls_list_str = ",".join(urls_list)

keywords = "gaza,israel,hamas,idf,palestine,war"

scraper = WebScraper2(sub_url_size = 3 , keywords = keywords)
                        # Integration with DB will make it faster in future, as fetching is much faster than scrapping.
inside_urls, failed_fetch, sub_url_size, total_size = scraper.get_suburls2(urls_list_str)

# print("Inside URLs:", inside_urls)
print("Failed Fetch:", failed_fetch)
print("Splits:", len(inside_urls))
print("Tree size:", total_size)

100%|██████████| 71/71 [00:21<00:00,  3.37it/s]

Failed Fetch: 0
Splits: 4
Tree size: 196


## Joining sub urls into one single list

In [5]:
website_urls = [item for sublist in list(inside_urls.values()) for item in sublist]
len(website_urls)
# print(len(website_urls))


196

## DB integration for Date

### Importing Date db from mongo

In [6]:
date_db_name = "PetraOil"
collection_db_name = "Date Database"
columns = ["url", "Date"]
mongo_date_df = import_from_mongo(date_db_name, collection_db_name, columns)
mongo_date_df

,url,Date
131,https://www.indiatoday.in/undefined/world/stor...,2024-01-09
145,https://www.indiatoday.in/newsmo/video/israel-...,2024-01-09
130,https://www.indiatoday.in/world/story/us-secre...,2024-01-09
144,https://www.indiatoday.in/world/story/houthi-l...,2024-01-09
143,https://www.indiatoday.in/undefined/world/stor...,2024-01-09
...,...,...
108,https://www.indiatoday.in/world/story/rockets-...,2023-10-07
1,https://www.indiatoday.in/topic/israel,2019-09-12
12,https://www.indiatoday.in,2019-09-12
86,https://www.khaleejtimes.com/opinion/life-is-w...,2017-07-31


### Urls of extracted sub urls

In [7]:
urls_from_extraction = pd.DataFrame(website_urls[:]) 
urls_from_extraction.columns = ["url"]
urls_from_extraction

,url
0,https://www.khaleejtimes.com
1,https://www.indiatoday.in
2,https://www.indiatoday.in/topic/israel
3,https://www.indiatoday.in/sports/cricket/video...
4,https://www.khaleejtimes.com/long-reads/from-i...
...,...
191,https://www.khaleejtimes.com/world/mena/if-i-m...
192,https://www.indiatoday.in/undefined/india/stor...
193,https://www.indiatoday.in/india/video/bjp-hind...
194,https://www.indiatoday.in/undefined/world/stor...


### Urls that arent yet indexed in db

In [8]:
df_of_which_to_find_the_date_of = pd.merge(mongo_date_df, urls_from_extraction, how = "outer")
df_of_which_to_find_the_date_of = df_of_which_to_find_the_date_of.loc[~df_of_which_to_find_the_date_of['Date'].notna(), :] # Dates that are yet found out
df_of_which_to_find_the_date_of = df_of_which_to_find_the_date_of.drop_duplicates(subset='url', keep='first')              # Dropping duplicates if any
df_of_which_to_find_the_date_of

,url,Date
195,https://www.indiatoday.in/sports/cricket/video...,NaT
198,https://www.indiatoday.in/world/story/2023-pla...,NaT
199,https://www.indiatoday.in/undefined/world/stor...,NaT
201,https://www.indiatoday.in/sports/cricket/story...,NaT
203,https://www.indiatoday.in/world/story/uae-pres...,NaT
205,https://www.khaleejtimes.com/long-reads/a-sunk...,NaT
207,https://www.khaleejtimes.com/long-reads/book-b...,NaT
209,https://www.indiatoday.in/world/story/us-shoot...,NaT
210,https://www.khaleejtimes.com/world/japan-lower...,NaT
211,https://www.aa.com.tr/en/europe/irish-oppositi...,NaT


In [9]:
print(f"Yet to index: {df_of_which_to_find_the_date_of.shape[0]}")

Yet to index: 18


In [10]:
list_of_which_to_find_the_date_of = list(df_of_which_to_find_the_date_of["url"].values)
list_of_which_to_find_the_date_of

['https://www.indiatoday.in/sports/cricket/video/mohammed-shami-arjuna-award-biggest-achievement-of-my-life-2486252-2024-01-09',
 'https://www.indiatoday.in/world/story/2023-planet-hottest-on-record-warming-limit-2486474-2024-01-09?utm_source=directhp&utm_medium=clicktopstories&utm_campaign=hptopstories',
 'https://www.indiatoday.in/undefined/world/story/uae-president-sheikh-mohamed-bin-zayed-al-nahyan-israel-aid-for-palestinians-hamas-war-2486380-2024-01-09',
 'https://www.indiatoday.in/sports/cricket/story/mohammed-shami-arjuna-award-president-droupadi-murmu-national-sports-2486246-2024-01-09',
 'https://www.indiatoday.in/world/story/uae-president-sheikh-mohamed-bin-zayed-al-nahyan-israel-aid-for-palestinians-hamas-war-2486380-2024-01-09',
 'https://www.khaleejtimes.com/long-reads/a-sunken-submarine-in-uae-how-this-world-war-ii-u-boat-got-near-fujairah-shores',
 'https://www.khaleejtimes.com/long-reads/book-bans-chatbots-pedagogical-warfare-what-it-means-to-read-has-become-a-minefiel

### Getting dates for extracted urls that arent indexed in DB

In [13]:
url_date_list = []
timeout_seconds = 5

for url in tqdm(list_of_which_to_find_the_date_of):
    start_time = time()
    try:
        # Your function to fetch the date from the URL
        date_value = fetch_date_from_url(url)
    except Exception as e:
        # Handle exceptions, e.g., print an error message or store a default value
        print(f"Error fetching date for {url}: {e}")
        date_value = None

    elapsed_time = time() - start_time

    # Append to the list if the operation took less than the timeout
    if elapsed_time < timeout_seconds:
        url_date_list.append([url, date_value])
    else:
        print(f"Skipping {url} due to timeout of {timeout_seconds}s")


 61%|██████    | 11/18 [00:08<00:07,  1.04s/it]

An error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 78%|███████▊  | 14/18 [00:10<00:02,  1.41it/s]

An error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 18/18 [00:10<00:00,  1.69it/s]


In [14]:
url_date_list

[['https://www.indiatoday.in/sports/cricket/video/mohammed-shami-arjuna-award-biggest-achievement-of-my-life-2486252-2024-01-09',
  '09-01-2024'],
 ['https://www.indiatoday.in/world/story/2023-planet-hottest-on-record-warming-limit-2486474-2024-01-09?utm_source=directhp&utm_medium=clicktopstories&utm_campaign=hptopstories',
  '09-01-2024'],
 ['https://www.indiatoday.in/undefined/world/story/uae-president-sheikh-mohamed-bin-zayed-al-nahyan-israel-aid-for-palestinians-hamas-war-2486380-2024-01-09',
  '09-01-2024'],
 ['https://www.indiatoday.in/sports/cricket/story/mohammed-shami-arjuna-award-president-droupadi-murmu-national-sports-2486246-2024-01-09',
  '09-01-2024'],
 ['https://www.indiatoday.in/world/story/uae-president-sheikh-mohamed-bin-zayed-al-nahyan-israel-aid-for-palestinians-hamas-war-2486380-2024-01-09',
  '09-01-2024'],
 ['https://www.khaleejtimes.com/long-reads/a-sunken-submarine-in-uae-how-this-world-war-ii-u-boat-got-near-fujairah-shores',
  '20-10-2023'],
 ['https://www.k

In [15]:
url_date_dict = {i[0] : i[1] for i in url_date_list}
url_date_dict

{'https://www.indiatoday.in/sports/cricket/video/mohammed-shami-arjuna-award-biggest-achievement-of-my-life-2486252-2024-01-09': '09-01-2024',
 'https://www.indiatoday.in/world/story/2023-planet-hottest-on-record-warming-limit-2486474-2024-01-09?utm_source=directhp&utm_medium=clicktopstories&utm_campaign=hptopstories': '09-01-2024',
 'https://www.indiatoday.in/undefined/world/story/uae-president-sheikh-mohamed-bin-zayed-al-nahyan-israel-aid-for-palestinians-hamas-war-2486380-2024-01-09': '09-01-2024',
 'https://www.indiatoday.in/sports/cricket/story/mohammed-shami-arjuna-award-president-droupadi-murmu-national-sports-2486246-2024-01-09': '09-01-2024',
 'https://www.indiatoday.in/world/story/uae-president-sheikh-mohamed-bin-zayed-al-nahyan-israel-aid-for-palestinians-hamas-war-2486380-2024-01-09': '09-01-2024',
 'https://www.khaleejtimes.com/long-reads/a-sunken-submarine-in-uae-how-this-world-war-ii-u-boat-got-near-fujairah-shores': '20-10-2023',
 'https://www.khaleejtimes.com/long-read

In [16]:
# Insert the document into the collection

data = url_date_dict
date_db_name = "PetraOil"
collection_db_name = "Date Database"


save_to_mongo(date_db_name, collection_db_name, data = data)

Inserted in Mongodb Cloud
Database: PetraOil
Collection: Date Database


In [17]:
urls_from_extraction

,url
0,https://www.khaleejtimes.com
1,https://www.indiatoday.in
2,https://www.indiatoday.in/topic/israel
3,https://www.indiatoday.in/sports/cricket/video...
4,https://www.khaleejtimes.com/long-reads/from-i...
...,...
191,https://www.khaleejtimes.com/world/mena/if-i-m...
192,https://www.indiatoday.in/undefined/india/stor...
193,https://www.indiatoday.in/india/video/bjp-hind...
194,https://www.indiatoday.in/undefined/world/stor...


In [18]:
urls_date_df = pd.merge(urls_from_extraction, mongo_date_df, on='url', how='inner').sort_values(by="Date",ascending=False)
urls_date_df

,url,Date
43,https://www.indiatoday.in/world/story/new-york...,2024-01-09
57,https://www.indiatoday.in/world/story/us-secre...,2024-01-09
27,https://www.indiatoday.in/world/story/houthi-l...,2024-01-09
28,https://www.indiatoday.in/world/story/fauda-ac...,2024-01-09
29,https://www.indiatoday.in/world/story/fauda-ac...,2024-01-09
...,...,...
1,https://www.indiatoday.in,2019-09-12
4,https://www.indiatoday.in/topic/israel,2019-09-12
3,https://www.indiatoday.in/topic/israel,2019-09-12
141,https://www.khaleejtimes.com/opinion/life-is-w...,2017-07-31


In [19]:
urls_date_df.isna().sum()

url     0
Date    0
dtype: int64

In [20]:
urls_date_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 171 entries, 43 to 147
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   url     171 non-null    object        
 1   Date    171 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 4.0+ KB


# Html content analysing

In [21]:
start_date = pd.to_datetime('06-01-2024',format='%d-%m-%Y')
end_date = pd.to_datetime('09-01-2024',format='%d-%m-%Y')

df_filtered_by_date = urls_date_df[(urls_date_df["Date"] >= start_date) & (urls_date_df["Date"] <= end_date)]
df_filtered_by_date

,url,Date
43,https://www.indiatoday.in/world/story/new-york...,2024-01-09
57,https://www.indiatoday.in/world/story/us-secre...,2024-01-09
27,https://www.indiatoday.in/world/story/houthi-l...,2024-01-09
28,https://www.indiatoday.in/world/story/fauda-ac...,2024-01-09
29,https://www.indiatoday.in/world/story/fauda-ac...,2024-01-09
...,...,...
30,https://www.indiatoday.in/undefined/world/stor...,2024-01-06
31,https://www.indiatoday.in/undefined/world/stor...,2024-01-06
58,https://www.indiatoday.in/undefined/world/stor...,2024-01-06
59,https://www.indiatoday.in/undefined/world/stor...,2024-01-06


## Creating {url : html content} dictionary

In [22]:
date_db_name = "PetraOil"
collection_db_name = "Html Database"
columns = ["url", "Html"]
mongo_html_df = import_from_mongo(date_db_name, collection_db_name, columns)
mongo_html_df

,url,Html
0,https://www.indiatoday.in/undefined/world/stor...,New York: Pro-Palestine protesters arrested fo...
1,https://www.indiatoday.in/world/story/fauda-ac...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
2,https://www.indiatoday.in/undefined/world/stor...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
3,https://www.indiatoday.in/undefined/world/stor...,US Secretary of State to meet Israeli leaders ...
4,https://www.indiatoday.in/world/story/war-isra...,"Israel says big arms unit located in Gaza, new..."
5,https://www.indiatoday.in/world/story/us-secre...,US Secretary of State to meet Israeli leaders ...
6,https://www.indiatoday.in/undefined/world/stor...,"Israel says big arms unit located in Gaza, new..."
7,https://www.indiatoday.in/world/story/new-york...,New York: Pro-Palestine protesters arrested fo...
8,https://www.indiatoday.in/undefined/world/stor...,Houthi leader Mohammed Ali al-Houthi's intervi...
9,https://www.indiatoday.in/world/story/houthi-l...,Houthi leader Mohammed Ali al-Houthi's intervi...


In [23]:
df_of_which_to_find_the_html_of = pd.merge(mongo_html_df, pd.DataFrame(df_filtered_by_date["url"]) , how = "outer")
df_of_which_to_find_the_html_of = df_of_which_to_find_the_html_of.loc[~df_of_which_to_find_the_html_of['Html'].notna(), :] # Dates that are yet found out
df_of_which_to_find_the_html_of = df_of_which_to_find_the_html_of.drop_duplicates(subset='url', keep='first')              # Dropping duplicates if any
df_of_which_to_find_the_html_of

,url,Html
82,https://www.khaleejtimes.com/uae/uae-president...,NaN


In [24]:
print(f"Yet to index: {df_of_which_to_find_the_html_of.shape[0]}")

Yet to index: 1


In [25]:
list_of_which_to_find_the_html_of = list(df_of_which_to_find_the_html_of["url"])
list_of_which_to_find_the_html_of

['https://www.khaleejtimes.com/uae/uae-president-us-secretary-of-state-discuss-serious-impact-of-gaza-crisis']

In [26]:
url_html_extracted,_ = get_html(list_of_which_to_find_the_html_of)
# url_html_extracted

100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


In [27]:
data = url_date_dict
date_db_name = "PetraOil"
collection_db_name = "Html Database"
save_to_mongo(date_db_name, collection_db_name, data = url_html_extracted)

Inserted in Mongodb Cloud
Database: PetraOil
Collection: Html Database


In [28]:
date_db_name = "PetraOil"
collection_db_name = "Html Database"
columns = ["url", "Html"]
mongo_html_df = import_from_mongo(date_db_name, collection_db_name, columns)
mongo_html_df

,url,Html
0,https://www.indiatoday.in/undefined/world/stor...,New York: Pro-Palestine protesters arrested fo...
1,https://www.indiatoday.in/world/story/fauda-ac...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
2,https://www.indiatoday.in/undefined/world/stor...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
3,https://www.indiatoday.in/undefined/world/stor...,US Secretary of State to meet Israeli leaders ...
4,https://www.indiatoday.in/world/story/war-isra...,"Israel says big arms unit located in Gaza, new..."
5,https://www.indiatoday.in/world/story/us-secre...,US Secretary of State to meet Israeli leaders ...
6,https://www.indiatoday.in/undefined/world/stor...,"Israel says big arms unit located in Gaza, new..."
7,https://www.indiatoday.in/world/story/new-york...,New York: Pro-Palestine protesters arrested fo...
8,https://www.indiatoday.in/undefined/world/stor...,Houthi leader Mohammed Ali al-Houthi's intervi...
9,https://www.indiatoday.in/world/story/houthi-l...,Houthi leader Mohammed Ali al-Houthi's intervi...


In [29]:
url_html_df_date_sorted = mongo_html_df[mongo_html_df['url'].isin(list(df_filtered_by_date["url"]))]
url_html_df_date_sorted

,url,Html
0,https://www.indiatoday.in/undefined/world/stor...,New York: Pro-Palestine protesters arrested fo...
1,https://www.indiatoday.in/world/story/fauda-ac...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
2,https://www.indiatoday.in/undefined/world/stor...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
3,https://www.indiatoday.in/undefined/world/stor...,US Secretary of State to meet Israeli leaders ...
4,https://www.indiatoday.in/world/story/war-isra...,"Israel says big arms unit located in Gaza, new..."
5,https://www.indiatoday.in/world/story/us-secre...,US Secretary of State to meet Israeli leaders ...
6,https://www.indiatoday.in/undefined/world/stor...,"Israel says big arms unit located in Gaza, new..."
7,https://www.indiatoday.in/world/story/new-york...,New York: Pro-Palestine protesters arrested fo...
8,https://www.indiatoday.in/undefined/world/stor...,Houthi leader Mohammed Ali al-Houthi's intervi...
9,https://www.indiatoday.in/world/story/houthi-l...,Houthi leader Mohammed Ali al-Houthi's intervi...


In [30]:
url_html_df_date_sorted_10 = url_html_df_date_sorted[:10]              # First 10 urls
url_html_df_date_sorted_10

,url,Html
0,https://www.indiatoday.in/undefined/world/stor...,New York: Pro-Palestine protesters arrested fo...
1,https://www.indiatoday.in/world/story/fauda-ac...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
2,https://www.indiatoday.in/undefined/world/stor...,Israel Hamas War: 'Fauda' actor Idan Amedi inj...
3,https://www.indiatoday.in/undefined/world/stor...,US Secretary of State to meet Israeli leaders ...
4,https://www.indiatoday.in/world/story/war-isra...,"Israel says big arms unit located in Gaza, new..."
5,https://www.indiatoday.in/world/story/us-secre...,US Secretary of State to meet Israeli leaders ...
6,https://www.indiatoday.in/undefined/world/stor...,"Israel says big arms unit located in Gaza, new..."
7,https://www.indiatoday.in/world/story/new-york...,New York: Pro-Palestine protesters arrested fo...
8,https://www.indiatoday.in/undefined/world/stor...,Houthi leader Mohammed Ali al-Houthi's intervi...
9,https://www.indiatoday.in/world/story/houthi-l...,Houthi leader Mohammed Ali al-Houthi's intervi...


In [ ]:
url_html_dict = url_html_df_date_sorted_10.set_index('url')['Html'].to_dict()
# url_html_dict

## Keyword extraction performed on above dictionary

In [32]:
try:
    del url_html_extracted["_id"]
    print("Deleted _id")
except:
    pass

Deleted _id


In [33]:
url_extracted_html = kep(website_content = url_html_dict, keywords = keywords, filter_by_amount = 60)

# url_extracted_html

100%|██████████| 10/10 [00:00<00:00, 5948.52it/s]


## Converting dictionary to list of tuple pairs, for implementation of batches

In [34]:
content_list = [(key,value[:2000]) for key, value in url_extracted_html.items()] # 1000 is temporary until tokenier function is not set up
# content_list

## Cutting the above list fo batches of batch size MAX_CONTENT

In [35]:
MAX_CONTENT = 5

content_list_complete = []

iterations = len(content_list) // MAX_CONTENT


for i in range(iterations):
    sub_content_list = content_list[MAX_CONTENT * i: MAX_CONTENT * (i + 1)]
    content_list_complete.append(sub_content_list)

# Handle remaining elements after the loop
remaining_elements = content_list[MAX_CONTENT * iterations:]
if remaining_elements:
    iterations += 1
    content_list_complete.append(remaining_elements)

len(content_list_complete)

2

## Openai's api

### Non parallel execution of 1 api

In [37]:
from time import sleep
start = time()

question = "Summary of situation in gaza"

response_complete = ''
for data_idx in range(iterations):

    prompt = f""" 
        Data is in the form of tuples inside list: {content_list_complete[data_idx]} \n\n\n 
        Question: {question} \n\n\n
        Method of reply: 100 - 200 word sentences, clear reply,
        provide url if neccessary.
        """
    
    if data_idx % 6 == 0:
        sleep(20)

    response = get_completion(prompt)
    response_complete += response + "\n\n"
    print(f"Batch {data_idx + 1} out of {iterations} completed ")

end = time()

print(f"Executed in {end-start:.2f}s")
    

Batch 1 out of 2 completed 
Batch 2 out of 2 completed 
Executed in 41.40s


In [38]:
response_complete

'The situation in Gaza remains tense as the Israel-Hamas conflict continues. Pro-Palestinian protesters in New York City have been arrested for blocking bridges and demanding an immediate ceasefire. The protests highlight the global concern for the ongoing violence and the need for a resolution to the conflict. In addition, Israeli actor Idan Amedi, known for his role in the show "Fauda," has been severely injured in the battle between the Israeli Defense Forces (IDF) and Hamas in Gaza\'s Khan Younis. This serves as a reminder of the human toll of the conflict. Meanwhile, US Secretary of State Antony Blinken is meeting with Israeli leaders to prevent the situation from escalating further. He is urging the Israeli government to do more to protect civilians and ensure humanitarian assistance reaches those in need. The Israeli military has also announced the discovery of a large arms unit in Gaza, further highlighting the ongoing threat posed by Hamas. The situation in Gaza remains a pres

### Parallel Execution for 2 api

In [ ]:
start = time()

complete_result_of_openai = ""
question_to_pass = "status of war in gaza"

count = 0

NUM_OF_API = 2

for i in range(0, len(content_list_complete), NUM_OF_API):

    if count % 6 == 0:
        sleep(20)
    

    result = ''

    result_queue1 = multiprocessing.Queue()
    result_queue2 = multiprocessing.Queue()


    process1 = multiprocessing.Process(target=gpt1, args=(question_to_pass, content_list_complete, i, result_queue1))
    try:
        process2 = multiprocessing.Process(target=gpt2, args=(question_to_pass, content_list_complete, i+1, result_queue2))
    except:
        pass


    # Start processes
    process1.start()
    process2.start()

    # Wait for processes to finish
    process1.join()
    process2.join()


    result1 = result_queue1.get()
    result2 = result_queue2.get()

    # Rest of your code remains unchanged
    result = result1 + "\n\n" + result2 + "\n\n"
    complete_result_of_openai += result

    print(f"Batch {i+1} - {i+NUM_OF_API} executed out of {len(content_list_complete)}")
    count += 1
complete_result_of_openai

end = time()

print(f"Executed in {end-start:.2f}s")


In [17]:
response2 = get_completion(f"Provide Detailed Summary of {complete_result_of_openai}")
# response3

In [23]:
with open("Output_gaza_parallel.txt" , "w") as f:
    f.write(complete_result_of_openai)

In [18]:
with open("Output_gaza_parallel_summary.txt" , "w") as f:
    f.write(complete_result_of_openai)

In [183]:
with open("Output_gaza_db.txt" , "w") as f:
    f.write(response_complete)

In [66]:
with open("Output_gaza_summary.txt" , "w") as f:
    f.write(response2)